- built-in range function
- property decorator
- namedtuple class constructor
- Enum class (since Python 3.4)
- dataclass (since Python 3.7)
- etc.

# Built-in `range` function

In [1]:
print(list(range(10)))
print(list(range(2, 10, 3)))
print(list(range(5, 0, -2)))
print(list(range(4, 1)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[2, 5, 8]
[5, 3, 1]
[]


In [2]:
def my_range(start, stop=None, step=1):
    """Implements built-in ``range()`` function."""
    
    if not isinstance(step, int):
        raise TypeError('step must be integer')
    if step == 0:
        raise ValueError('step cannot be zero')
    
    if stop is None:  # built-in syntax special case
        stop = start; start = 0    
    curr = start
    
    while (curr < stop) if (step > 0) else (curr > stop):
        yield curr
        curr += step


In [3]:
print(list(my_range(10)))
print(list(my_range(2, 10, 3)))
print(list(my_range(5, 0, -2)))
print(list(my_range(4, 1)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[2, 5, 8]
[5, 3, 1]
[]


# A Note About Descriptors

In [4]:
class PositiveNumber(object):
    
    def __init__(self, default=1):
        self.default = default
    
    def __set_name__(self, cls, name):
        self.attribute_name = name
        self.binding_cls = cls
    
    def __get__(self, instance, cls=None):
        if instance is None:
            return self
        return instance.__dict__[self.attribute_name]

    def __set__(self, instance, value):
        if value <= 0:
            raise ValueError('number must be positive')
        instance.__dict__[self.attribute_name] = value

In [5]:
class Person(object):
    name: str
    age: int = PositiveNumber()
        
    def __init__(self, name, age=1):
        self.name = name
        self.age = age
        
alice = Person("Alice")
print(alice.age)
alice.age = 10
print(alice.age)
#alice.age = -10

1
10


# Built-in `property` decorator function

In [6]:
class AspectRatioRectangle(object):
    """
    Rectangle object maintaining the original
    aspect ratio when resizing width or height.
    """
    
    def __init__(self, width, height):
        self.original_width = width
        self.original_height = height
        self.scale = 1
        
    @property
    def width(self):
        return self.original_width * self.scale
    
    @width.setter
    def width(self, new_width):
        self.scale = new_width / self.original_width
    
    @property
    def height(self):
        return self.original_height * self.scale
    
    @height.setter
    def height(self, new_height):
        self.scale = new_height / self.original_height
        
    def __repr__(self):
        return f"{type(self).__name__}({self.width}, {self.height})"

In [7]:
rect = AspectRatioRectangle(3, 4)
print(rect)
rect.height = 10
print(rect)
rect.width = 6
print(rect)

AspectRatioRectangle(3, 4)
AspectRatioRectangle(7.5, 10.0)
AspectRatioRectangle(6.0, 8.0)


In [8]:
class my_property(object):
    """
    Implements built-in ``property`` decorator function.
    Adapted from https://docs.python.org/3.6/howto/descriptor.html
    """
    def __init__(self, getter_fn):
        self.getter_fn = getter_fn
        self.setter_fn = None
        
    def __get__(self, instance, cls=None):
        if instance is None:
            return self          
        return self.getter_fn(instance)

    def __set__(self, instance, value):
        if self.setter_fn is None:
            raise AttributeError("cannot modify attribute")
        self.setter_fn(instance, value)

    def setter(self, setter_fn):
        self.setter_fn = setter_fn
        return self

In [9]:
class AspectRatioRectangle(object):
    
    def __init__(self, width, height):
        self.original_width = width
        self.original_height = height
        self.scale = 1
        
    @my_property
    def width(self):
        return self.original_width * self.scale
    
    @width.setter
    def width(self, new_width):
        self.scale = new_width / self.original_width
    
    @my_property
    def height(self):
        return self.original_height * self.scale
    
    @height.setter
    def height(self, new_height):
        self.scale = new_height / self.original_height
        
    def __repr__(self):
        return f"{type(self).__name__}({self.width}, {self.height})"

In [10]:
rect = AspectRatioRectangle(3, 4)
print(rect)
rect.height = 10
print(rect)
rect.width = 6
print(rect)

AspectRatioRectangle(3, 4)
AspectRatioRectangle(7.5, 10.0)
AspectRatioRectangle(6.0, 8.0)
